In [1]:
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph

In [2]:
load_dotenv(".env")

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


In [3]:
kg_db_name = "t20"
# kg_db_name = "t30documentsgraph"

In [4]:
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=kg_db_name
)

/var/folders/sq/lqj95by96lnccb74l6fsytf00000gn/T/ipykernel_59432/2508706621.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  kg = Neo4jGraph(


#### Detect Hierarchical communities

In [5]:
from src.hierarchical_community_detector import HierarchicalCommunityDetector

In [6]:
hierarchical_community_detector = HierarchicalCommunityDetector(
    kg=kg,
    kg_db_name=kg_db_name)

In [7]:
hierarchical_community_detector.detect_hierarchical_communities()


▶ Base graph 't20_projection' already projected.

  Skipping Leiden on t20_projection: C3_CommunityId already exists.

▶ Graph 'C3_graph' already projected.

▶ Running Leiden on C3_graph, writing C2_CommunityId...

✅ Full C3 → C0 community hierarchy generated.


#### Old code 

In [5]:
cypher = """
CALL gds.graph.list()
"""

In [6]:
graphs =  kg.query(query=cypher)
print(len(graphs))

3


In [7]:
graphs_db = [graph["database"] for graph in graphs]

In [8]:
# kg_db_name = "t30documentsgraph"
if kg_db_name in graphs_db:
    print("There exist a Graph associated to the database")
    
else:
    print("There is not a Graph associated to the database")
    print("Creating Graph in memory")
    labels = kg.query(query=
        """
        CALL db.labels() YIELD label
        RETURN collect(label) AS allLabels;
        """
    )[0]["allLabels"]
    
    relationships = kg.query(query=
        """
        CALL db.relationshipTypes() YIELD relationshipType
        RETURN collect(relationshipType) AS allTypes;
        """
    )[0]["allTypes"]

    project_query = f"""
    CALL gds.graph.project(
    '{kg_db_name}',
    {labels},
    {relationships}
    )
    """

    # project the database in a graph 
    kg.query(query=project_query)

    print("Graph created succesfully")


There is not a Graph associated to the database
Creating Graph in memory
Graph created succesfully


In [ ]:
# we can drop a graph in the following way 
# CALL gds.graph.drop('myGraphProjection');
# it should be executed in the database that contain the graph projected in memory

#### <font color="yellow"> Identify communities and write back resutl into the database

In [9]:
# Run Leiben algorithm to identify communities in the in memory graph, and write 
# to each node the property community id 

# kg_db_name = "t30documentsgraph"

leiben_property_cypher = f"""
    CALL gds.alpha.leiden.write(
    '{kg_db_name}',
    {{writeProperty: 'communityId' }}
    )
    YIELD communityCount, modularity;
"""

kg.query(query=leiben_property_cypher)



[{'communityCount': 63, 'modularity': 0.6137139162605811}]

In [10]:
community_id_cypher = """
MATCH (n)
WHERE exists(n.communityId)
RETURN n.communityId AS communityId, count(*) AS nodesCount
ORDER BY communityId
"""

community_ids = [com['communityId'] for com in kg.query(query=community_id_cypher)]
print(community_ids)
print(len(community_ids))


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: The property existence syntax `... exists(variable.property)` is deprecated, please use `variable.property IS NOT NULL` instead} {position: line: 3, column: 7, offset: 17} for query: '\nMATCH (n)\nWHERE exists(n.communityId)\nRETURN n.communityId AS communityId, count(*) AS nodesCount\nORDER BY communityId\n'


[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 60, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 74, 76, 79]
63


#### Get Nodes in Each Community

In [72]:
community_nodes_cypher = """
MATCH (n)
WHERE exists(n.communityId)
RETURN n.communityId AS communityId, collect(id(n)) AS nodeIds
"""
community_nodes = {res['communityId']: res['nodeIds'] for res in kg.query(community_nodes_cypher)}

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: The property existence syntax `... exists(variable.property)` is deprecated, please use `variable.property IS NOT NULL` instead} {position: line: 3, column: 7, offset: 17} for query: '\nMATCH (n)\nWHERE exists(n.communityId)\nRETURN n.communityId AS communityId, collect(id(n)) AS nodeIds\n'


In [73]:
print(community_nodes)

{22: [0, 1, 2, 3, 4, 5, 43, 86, 87, 88, 89, 90, 91, 92, 93, 140, 141, 142, 145], 16: [6, 7, 8, 9, 10, 11, 12], 9: [13, 14, 15, 16, 17, 18, 21, 101, 102, 103, 105, 106, 107, 108], 20: [19], 46: [20], 0: [22, 23, 24, 25, 27], 8: [26], 14: [28, 29, 30, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 48], 15: [31, 32, 33, 46, 47, 49, 50, 51, 52, 75, 76, 77, 78, 79, 80, 81, 132, 133, 134, 135, 136, 137, 138, 139], 37: [45], 17: [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 82, 83, 84], 29: [63, 64, 65, 66, 69, 70], 2: [67], 35: [68], 10: [71, 72, 73, 74], 31: [85], 41: [94, 95, 96, 97, 98, 99, 100], 33: [104], 40: [109, 110, 111, 112, 116, 117, 118], 43: [113, 114, 115], 1: [119, 123, 124, 125, 126, 127, 128, 129, 130, 131], 21: [120, 121, 122], 38: [143], 3: [144], 4: [146, 147, 148, 149, 150, 151, 155, 158, 160, 161, 162, 163, 165], 12: [152], 5: [153], 28: [154], 7: [156], 27: [157], 19: [159], 13: [164], 6: [166]}


In [ ]:
# for each node community get the ranked edge 

#### Needed setup variables 

In [1]:
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph

In [2]:
load_dotenv(".env")

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


In [3]:
kg_db_name = "t20"
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=kg_db_name
)

/var/folders/sq/lqj95by96lnccb74l6fsytf00000gn/T/ipykernel_53989/788261938.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  kg = Neo4jGraph(


#### Test Community detection for different level 

In [4]:
from src.hierarchical_community_detector import HierarchicalCommunityDetector

In [5]:
detector = HierarchicalCommunityDetector(kg=kg, kg_db_name=kg_db_name)

In [6]:
detector.detect_hierarchical_communities()


▶ Base graph 't20_projection' already projected.

  Skipping Leiden on t20_projection: C3_CommunityId already exists.

▶ Summarizing C3 Community 6...
✅ Community Summary already esist for Community 6

▶ Summarizing C3 Community 7...
✅ Community Summary already esist for Community 7

▶ Summarizing C3 Community 42...
✅ Community Summary already esist for Community 42

▶ Summarizing C3 Community 64...
✅ Community Summary already esist for Community 64

▶ Summarizing C3 Community 14...
✅ Community Summary already esist for Community 14

▶ Summarizing C3 Community 15...
✅ Community Summary already esist for Community 15

▶ Summarizing C3 Community 62...
✅ Community Summary already esist for Community 62

▶ Summarizing C3 Community 4...
✅ Community Summary already esist for Community 4

▶ Summarizing C3 Community 70...
✅ Community Summary already esist for Community 70

▶ Summarizing C3 Community 1...
✅ Community Summary already esist for Community 1

▶ Summarizing C3 Community 21...
✅ Com

In [6]:
hierarchycal_structure = detector.build_community_hierarchy_flexible()
print(hierarchycal_structure)

🔍 Checking available community levels...
✅ C2 Level: Yes
✅ C1 Level: No
✅ C0 Level: No
[{'C2_CommunityId': 14, 'children': [6]}, {'C2_CommunityId': 27, 'children': [7, 64, 0, 2]}, {'C2_CommunityId': 13, 'children': [42]}, {'C2_CommunityId': 8, 'children': [14]}, {'C2_CommunityId': 11, 'children': [15]}, {'C2_CommunityId': 10, 'children': [62]}, {'C2_CommunityId': 19, 'children': [4]}, {'C2_CommunityId': 18, 'children': [70]}, {'C2_CommunityId': 21, 'children': [1]}, {'C2_CommunityId': 20, 'children': [21]}, {'C2_CommunityId': 17, 'children': [18]}, {'C2_CommunityId': 15, 'children': [9]}, {'C2_CommunityId': 16, 'children': [22]}, {'C2_CommunityId': 28, 'children': [16]}, {'C2_CommunityId': 26, 'children': [17]}, {'C2_CommunityId': 23, 'children': [37]}, {'C2_CommunityId': 22, 'children': [39]}, {'C2_CommunityId': 25, 'children': [60]}, {'C2_CommunityId': 24, 'children': [48]}, {'C2_CommunityId': 7, 'children': [5]}, {'C2_CommunityId': 6, 'children': [10]}, {'C2_CommunityId': 4, 'childr

In [8]:
print(len(hierarchycal_structure))

27


### Test CommunitySummaries 

In [ ]:
from src.leaf_level_community_summaries import create_leaf_level_community_summaries

In [ ]:
non_useful_communities_ids = create_leaf_level_community_summaries(
    kg=kg,
    kg_db_name=kg_db_name
)


▶ Summarizing C3 Community 6...
✅ Community Summary already esist for Community 6

▶ Summarizing C3 Community 7...
✅ Community Summary already esist for Community 7

▶ Summarizing C3 Community 42...
✅ Community Summary already esist for Community 42

▶ Summarizing C3 Community 64...
✅ Community Summary already esist for Community 64

▶ Summarizing C3 Community 55...
⚠️ Skipping Community 55: Insufficient data.

▶ Summarizing C3 Community 52...
⚠️ Skipping Community 52: Insufficient data.

▶ Summarizing C3 Community 13...
⚠️ Skipping Community 13: Insufficient data.

▶ Summarizing C3 Community 14...
✅ Community Summary already esist for Community 14

▶ Summarizing C3 Community 15...
✅ Community Summary already esist for Community 15

▶ Summarizing C3 Community 62...
✅ Community Summary already esist for Community 62

▶ Summarizing C3 Community 4...
✅ Community Summary already esist for Community 4

▶ Summarizing C3 Community 70...
✅ Community Summary already esist for Community 70

▶ S

In [17]:
non_useful_communities_ids.sort()
print(non_useful_communities_ids)

[3, 11, 12, 13, 19, 24, 25, 26, 27, 28, 29, 30, 33, 36, 38, 40, 41, 43, 45, 47, 51, 52, 53, 55, 59, 61, 66, 67, 72, 75, 78]


#### Test hierarchicla construction

#### High level community summaries

In [1]:
from src.higher_level_community_summaries import HighLevelCommunitySummarizer

In [10]:
summarizer = HighLevelCommunitySummarizer(
    hierarchy=hierarchycal_structure,
    summary_root_dir="/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20",
    model="o4-mini-2025-04-16"
)

summarizer.run()


📘 Processing summaries at level: C2 (27 communities)
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_14.json
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_27.json
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_13.json
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_8.json
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_11.json
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_10.json
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_19.json
✅ Saved high-level summary: /Users/danielguarnizo/workspace/Master/LLMs/GraphRA

In [8]:
summarizer = HighLevelCommunitySummarizer(
    hierarchy=hierarchycal_structure,
    summary_root_dir="/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20",
    model="o4-mini-2025-04-16"
)

summarizer.run()


📘 Processing summaries at level: C2 (27 communities)
⏭️ Skipping C2_14, summary already exists at: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_14.json
⏭️ Skipping C2_27, summary already exists at: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_27.json
⏭️ Skipping C2_13, summary already exists at: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_13.json
⏭️ Skipping C2_8, summary already exists at: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_8.json
⏭️ Skipping C2_11, summary already exists at: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_11.json
⏭️ Skipping C2_10, summary already exists at: /Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2/C2_10.json
⏭️ Skipping C2_19, summary already exists at: /Users/danielguarnizo/workspace/Master/LLMs/GraphR

#### Test Community Answer

In [1]:
from src.global_query_answerer import GlobalQueryAnswerer

In [11]:
c3_community_anwer = GlobalQueryAnswerer(
    max_tokens= 200000, #TODO check which is the most suitable mac tokesn
    summary_dir="/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C3",
    model_name="o4-mini-2025-04-16"
)

c3_community_anwer.answer_query(
    query="What are the most commond methods used in papers discussing multi_agent systems",
)

🧩 Prepared 1 summary chunks.
✅ Collected 1 helpful answers.


'Below is a consolidated overview of the methods used in the reviewed multi-agent system (MAS) literature, organized into seven broad categories and highlighting their relative prominence and roles.\n\n1. Core MAS Principles and Concepts  \n   • Autonomy, proactivity and local decision-making  \n   • Agent communication languages / protocols (ACL, FIPA-ACL)  \n   • Organizational models (hierarchies, holarchies, societies)  \n   • Underpinning theory (BDI, blackboard, tuple-spaces)  \n   – Most frequently cited foundation, appearing in the majority of papers as the lens through which all other techniques are deployed.\n\n2. Coordination Mechanisms  \n   • Task allocation protocols (Contract Net, auction/market-based schemes)  \n   • Negotiation and argumentation frameworks  \n   • Shared plans and joint‐intention models  \n   – Key to synchronizing distributed agents, ensuring conflict resolution and workload balancing.\n\n3. Swarm Intelligence  \n   • Bio-inspired algorithms (Ant Colo

In [7]:
answer_1 = "'Below is a consolidated overview of the methods used in the reviewed multi-agent system (MAS) literature, organized into seven broad categories and highlighting their relative prominence and roles.\n\n1. Core MAS Principles and Concepts  \n   • Autonomy, proactivity and local decision-making  \n   • Agent communication languages / protocols (ACL, FIPA-ACL)  \n   • Organizational models (hierarchies, holarchies, societies)  \n   • Underpinning theory (BDI, blackboard, tuple-spaces)  \n   – Most frequently cited foundation, appearing in the majority of papers as the lens through which all other techniques are deployed.\n\n2. Coordination Mechanisms  \n   • Task allocation protocols (Contract Net, auction/market-based schemes)  \n   • Negotiation and argumentation frameworks  \n   • Shared plans and joint‐intention models  \n   – Key to synchronizing distributed agents, ensuring conflict resolution and workload balancing.\n\n3. Swarm Intelligence  \n   • Bio-inspired algorithms (Ant Colony Optimization, Particle Swarm Optimization)  \n   • Decentralized stigmergy and pheromone‐like information sharing  \n   – Exploits simple agent rules for emergent, scalable behaviors in routing, coverage and optimization tasks.\n\n4. Cooperation Models  \n   • Coalition formation and team planning  \n   • Shared beliefs/goals and commitment protocols  \n   • Trust and reputation systems for partner selection  \n   – Enables complex collaborative workflows, resilient group problem solving and adaptive task re-grouping.\n\n5. Delegation Mechanisms  \n   • Broker or manager agents that reassign subtasks  \n   • Dynamic role-assignment and capability matching  \n   • Delegation chains for multi-tiered task breakdown  \n   – Optimizes resource use, abstracts complexity and supports dynamic real-time reconfiguration.\n\n6. Companion (User-Centric) Agents  \n   • Adaptive interfaces that learn from user behavior  \n   • Context-awareness, personalization and recommendation engines  \n   – Bridges human–MAS interaction, often leveraging machine learning to tailor agent behavior.\n\n7. Mood and Affective Systems  \n   • Emotion models embedded in BDI architectures  \n   • Mood propagation across agent societies  \n   • Affective feedback loops to modulate decision-making  \n   – Improves believability, robustness under uncertainty and human-agent rapport.\n\nSummary of Prominence  \n– Multi-Agent System Principles/Concepts provide the structural and theoretical backbone in roughly 70–80% of the studies.  \n– Coordination and Swarm Intelligence follow, each appearing in around 40–60% of the papers.  \n– Cooperation and Delegation Mechanisms are less frequent (20–40%) but crucial for complex task domains.  \n– Companion Development and Mood Systems are emerging trends (under 20%) focused on human-centric and affective MAS.\n\nTaken together, these seven method classes form a layered toolkit: foundational MAS concepts enable coordination and cooperation protocols, which in turn can be enriched by swarm techniques, delegation layers, user-focused companion features and affective dimensions. Future work may explore tighter integration—e.g. embedding emotion-aware swarm agents or market-based delegation brokers—to push MAS adaptability, scalability and human-agent synergy even further.'"
print(answer_1)

'Below is a consolidated overview of the methods used in the reviewed multi-agent system (MAS) literature, organized into seven broad categories and highlighting their relative prominence and roles.

1. Core MAS Principles and Concepts  
   • Autonomy, proactivity and local decision-making  
   • Agent communication languages / protocols (ACL, FIPA-ACL)  
   • Organizational models (hierarchies, holarchies, societies)  
   • Underpinning theory (BDI, blackboard, tuple-spaces)  
   – Most frequently cited foundation, appearing in the majority of papers as the lens through which all other techniques are deployed.

2. Coordination Mechanisms  
   • Task allocation protocols (Contract Net, auction/market-based schemes)  
   • Negotiation and argumentation frameworks  
   • Shared plans and joint‐intention models  
   – Key to synchronizing distributed agents, ensuring conflict resolution and workload balancing.

3. Swarm Intelligence  
   • Bio-inspired algorithms (Ant Colony Optimization,

In [2]:
c3_community_anwer = GlobalQueryAnswerer(
    max_tokens= 200000, #TODO check which is the most suitable mac tokesn
    summary_dir="/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C3",
    model_name="o4-mini-2025-04-16"
)

c3_community_anwer.answer_query(
    query="What are the most commond methods used in papers discussing multi_agent systems",
)

🧩 Prepared 30 summary chunks.
✅ Collected 26 helpful answers.


'Across the multi‐agent systems literature, one can distill six broad—and frequently co‐occurring—methodological families.  Within each family sit a variety of concrete algorithms, frameworks and engineering practices.  \n\n1. Reinforcement-Learning and Game-Theoretic Approaches  \n   • Multi-agent RL: independent Q-learning, Deep Deterministic Policy Gradient (MADDPG), actor-critic and policy-gradient variants  \n   • Game-theoretic analysis: Nash‐equilibrium computation, fictitious play, strategic bargaining (e.g. Nash-bargaining protocols)  \n   • Tree-search methods: Monte-Carlo Tree Search (MCTS) when agents face combinatorial or game‐like decision spaces  \n\n2. Distributed Optimization and Task-Allocation  \n   • Distributed Constraint Optimization (DCOP) and related message-passing solvers  \n   • Auction- and market-based allocation (incl. Contract Net Protocols)  \n   • Decoupled modular architectures for negotiation (e.g. modular bargaining frameworks)  \n\n3. Coordination, 

In [ ]:
answer= "'Across the multi‐agent systems literature, one can distill six broad—and frequently co‐occurring—methodological families.  Within each family sit a variety of concrete algorithms, frameworks and engineering practices.  \n\n1. Reinforcement-Learning and Game-Theoretic Approaches  \n   • Multi-agent RL: independent Q-learning, Deep Deterministic Policy Gradient (MADDPG), actor-critic and policy-gradient variants  \n   • Game-theoretic analysis: Nash‐equilibrium computation, fictitious play, strategic bargaining (e.g. Nash-bargaining protocols)  \n   • Tree-search methods: Monte-Carlo Tree Search (MCTS) when agents face combinatorial or game‐like decision spaces  \n\n2. Distributed Optimization and Task-Allocation  \n   • Distributed Constraint Optimization (DCOP) and related message-passing solvers  \n   • Auction- and market-based allocation (incl. Contract Net Protocols)  \n   • Decoupled modular architectures for negotiation (e.g. modular bargaining frameworks)  \n\n3. Coordination, Communication and Consensus  \n   • Explicit coordination protocols: negotiation schemes, delegation mechanisms, contract nets  \n   • Consensus and agreement algorithms: flooding, gossip, leader-election, voting schemes  \n   • Multi-Agent Discussion frameworks and “Conquer-and-Merge” styles of iterative consensus  \n\n4. Agent-Based Modeling and Simulation  \n   • Agent-Based Modeling (ABM) toolkits for large‐scale simulation design  \n   • Scenario and environment generation driven by structured datasets  \n   • Integration of empirical testing (e.g. drive tests, user‐in‐the‐loop experiments)  \n\n5. LLM-Empowered and Prompt-Based Techniques  \n   • Prompt engineering variants: few-shot/example prompting, chain-of-thought prompting, iterative reflection  \n   • Retrieval-Augmented Generation (RAG) and external memory modules to pull in context at runtime  \n   • Question-Answering and dialogue‐style agent APIs built on frozen or fine-tuned pretrained LLMs  \n   • Hybrid workflows that generate or configure ABM/simulation code from LLM outputs  \n\n6. System-Engineering and Hybrid Architectures  \n   • Modular, plug-and-play agent components (e.g. “prompt agents,” knowledge-gradient policies, embedded knowledge graphs)  \n   • Model-centric vs. data-centric vs. engineering-centric emphases: from symbolic optimization or gradient-based fine-tuning to data curation and end-to-end system integration  \n   • Soliciting human feedback loops, empirical drive testing or crowd-sourced evaluation to close the design loop  \n\nTaken together, most modern multi-agent papers combine one or more algorithms from categories 1–3, embed them in ABM or empirical testbeds (category 4), and often bolster them with LLM-based scaffolding or prompt-engineering techniques (category 5), all within a carefully engineered software stack (category 6).'"

print(answer)

'Across the multi‐agent systems literature, one can distill six broad—and frequently co‐occurring—methodological families.  Within each family sit a variety of concrete algorithms, frameworks and engineering practices.  

1. Reinforcement-Learning and Game-Theoretic Approaches  
   • Multi-agent RL: independent Q-learning, Deep Deterministic Policy Gradient (MADDPG), actor-critic and policy-gradient variants  
   • Game-theoretic analysis: Nash‐equilibrium computation, fictitious play, strategic bargaining (e.g. Nash-bargaining protocols)  
   • Tree-search methods: Monte-Carlo Tree Search (MCTS) when agents face combinatorial or game‐like decision spaces  

2. Distributed Optimization and Task-Allocation  
   • Distributed Constraint Optimization (DCOP) and related message-passing solvers  
   • Auction- and market-based allocation (incl. Contract Net Protocols)  
   • Decoupled modular architectures for negotiation (e.g. modular bargaining frameworks)  

3. Coordination, Communicatio

In [ ]:
c2_community_anwer = GlobalQueryAnswerer(
    max_tokens= 200000, #TODO check which is the most suitable mac tokesn
    summary_dir="/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2",
    model_name="o4-mini-2025-04-16"
)


c2_community_anwer.answer_query(
    query="What are the most commond methods used in papers discussing multi_agent systems",
)
# 2m 21.9s

🧩 Prepared 27 summary chunks.
✅ Collected 23 helpful answers.


'Across the many communities and papers on multi-agent systems, a handful of methodological families repeatedly surface.  Below is a synthesis of the most commonly employed techniques, organized by broad category:\n\n1. Learning-Based Methods  \n  • Multi-Agent Reinforcement Learning (MARL)  \n    – Independent and Centralized Q-Learning, Policy-Gradient and Actor-Critic variants, Multi-Agent DQN  \n    – Hindsight Experience Replay (HER) in multi-agent settings  \n  • Hybrid Neuro-Symbolic & Symbolic Learning  \n    – Symbolic optimizers operating on high-level decision traces  \n    – Gradient-based neural fine-tuning + internal probing of attention patterns  \n    – Self-Evolving Agents using symbolic-rule modules alongside back-propagation  \n\n2. Game-Theoretic & Economic Models  \n  • Auction and Bargaining Mechanisms for task allocation  \n  • Nash and Correlated Equilibrium analyses  \n  • Contract-Net Protocols and Negotiation Heuristics  \n\n3. Distributed Optimization & Cons

In [4]:
c2_answer = "'Across the many communities and papers on multi-agent systems, a handful of methodological families repeatedly surface.  Below is a synthesis of the most commonly employed techniques, organized by broad category:\n\n1. Learning-Based Methods  \n  • Multi-Agent Reinforcement Learning (MARL)  \n    – Independent and Centralized Q-Learning, Policy-Gradient and Actor-Critic variants, Multi-Agent DQN  \n    – Hindsight Experience Replay (HER) in multi-agent settings  \n  • Hybrid Neuro-Symbolic & Symbolic Learning  \n    – Symbolic optimizers operating on high-level decision traces  \n    – Gradient-based neural fine-tuning + internal probing of attention patterns  \n    – Self-Evolving Agents using symbolic-rule modules alongside back-propagation  \n\n2. Game-Theoretic & Economic Models  \n  • Auction and Bargaining Mechanisms for task allocation  \n  • Nash and Correlated Equilibrium analyses  \n  • Contract-Net Protocols and Negotiation Heuristics  \n\n3. Distributed Optimization & Consensus  \n  • ADMM, Gossip- and Consensus-based algorithms for global objective alignment  \n  • Distributed Constraint Optimization Problems (DCOP)  \n  • Combinatorial and Resource-Allocation Workflows (routing, scheduling)  \n\n4. Coordination & Communication Schemas  \n  • Modular Architectures (“Conquer-and-Merge,” Decoupled Bargaining Modules)  \n  • Delegation Mechanisms: automated task hand-off among specialist agents  \n  • Swarm Intelligence and Bio-Inspired Heuristics (e.g. ant-colony, particle-swarm)  \n  • Emergent Communication Protocols, QA‐Driven Pipelines for ABM generation  \n\n5. Planning, Search & Simulation  \n  • Decentralized Monte Carlo Tree Search (MCTS) and POMDP Solvers  \n  • PDDL-Based Symbolic Planning integrated into agent workflows  \n  • Agent-Based Modeling Platforms (Repast, MASON) for large-scale scenario testing  \n\n6. Prompting & LLM-Centric Techniques  \n  • Prompt Engineering Frameworks: in-context learning, chain-of-thought, multi-objective prompts  \n  • Reflection and Self-Evaluation Loops—agents review and correct their own outputs  \n  • Retrieval-Augmented Generation (RAG) to ground agents in external knowledge during dynamic routing, resource allocation, and fault diagnosis  \n  • QA-Driven or ChatGPT-Style Pipelines, including feedback-driven reprompting and tool-calling agents  \n\n7. Novel Cooperative Architectures  \n  • Cooperative Multi-Agent Systems with specialized LLM agents (e.g. separate modules for geography, coverage optimization, cost trade-offs)  \n  • PromptAgentMethod: trial-and-error exploration, error feedback loops, Monte Carlo search for prompt optimization  \n  • “Companion” and “Mood” systems modeling narrative assistance and emotional state tracking across conversational agents  \n\nTaken together, these methods are often combined—for example, a MARL framework may employ RAG for rich observations, invoke auction protocols for resource negotiation, and leverage prompt-engineering heuristics for reasoning chains—yielding robust, flexible multi-agent solutions in domains from network optimization to narrative generation.'"

print(c2_answer)

'Across the many communities and papers on multi-agent systems, a handful of methodological families repeatedly surface.  Below is a synthesis of the most commonly employed techniques, organized by broad category:

1. Learning-Based Methods  
  • Multi-Agent Reinforcement Learning (MARL)  
    – Independent and Centralized Q-Learning, Policy-Gradient and Actor-Critic variants, Multi-Agent DQN  
    – Hindsight Experience Replay (HER) in multi-agent settings  
  • Hybrid Neuro-Symbolic & Symbolic Learning  
    – Symbolic optimizers operating on high-level decision traces  
    – Gradient-based neural fine-tuning + internal probing of attention patterns  
    – Self-Evolving Agents using symbolic-rule modules alongside back-propagation  

2. Game-Theoretic & Economic Models  
  • Auction and Bargaining Mechanisms for task allocation  
  • Nash and Correlated Equilibrium analyses  
  • Contract-Net Protocols and Negotiation Heuristics  

3. Distributed Optimization & Consensus  
  • ADMM

#### same execution but with parallel and prompts updates 

In [2]:
c2_community_anwer = GlobalQueryAnswerer(
    max_tokens= 200000, #TODO check which is the most suitable mac tokesn
    summary_dir="/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2",
    model_name="o4-mini-2025-04-16"
)


c2_community_anwer.answer_query(
    query="What are the most commond methods used in papers discussing multi_agent systems",
)


🧩 Prepared 27 summary chunks.
✅ Collected 20 helpful answers.


'# Common Methods in Multi-Agent Systems Research\n\nSynthesizing insights across multiple analyst reports, we identify six principal families of methods employed in multi-agent systems (MAS) papers. Each reflects distinct goals—ranging from prompt coordination and optimization loops to architectural patterns and domain-specific systems.\n\n---\n\n## 1. Prompt Engineering & In-Context Learning  \nRigorously defines how agents generate, structure, and share prompts to coordinate actions.  \n- **Formal Prompt Engineering Framework**: Specifies prompt spaces, constraints, and coordination objectives.  \n- **Multi-Objective Directional Prompting**: Guides each agent toward goals (e.g., accuracy, style) via directional vectors in prompt space.  \n- **Dynamic Prompt Assembly**: Constructs modular “sub-agent” prompts at inference time based on feedback and roles.  \n(Data: 7, 14, 117)\n\n## 2. Chain-of-Thought Reasoning & Reflection  \nStructures complex decision processes and self-evaluation

In [3]:
print("'# Common Methods in Multi-Agent Systems Research\n\nSynthesizing insights across multiple analyst reports, we identify six principal families of methods employed in multi-agent systems (MAS) papers. Each reflects distinct goals—ranging from prompt coordination and optimization loops to architectural patterns and domain-specific systems.\n\n---\n\n## 1. Prompt Engineering & In-Context Learning  \nRigorously defines how agents generate, structure, and share prompts to coordinate actions.  \n- **Formal Prompt Engineering Framework**: Specifies prompt spaces, constraints, and coordination objectives.  \n- **Multi-Objective Directional Prompting**: Guides each agent toward goals (e.g., accuracy, style) via directional vectors in prompt space.  \n- **Dynamic Prompt Assembly**: Constructs modular “sub-agent” prompts at inference time based on feedback and roles.  \n(Data: 7, 14, 117)\n\n## 2. Chain-of-Thought Reasoning & Reflection  \nStructures complex decision processes and self-evaluation across collaborating agents.  \n- **Chain-of-Thought**: Decomposes tasks into step-by-step reasoning shared among agents.  \n- **Reflection Loops**: Agents critique their own or peers’ outputs to refine subsequent iterations.  \n(Data: 14, 117)\n\n## 3. Coordination & Delegation Mechanisms  \nFocuses on how agents distribute tasks, resolve conflicts, and maintain coherent collaboration.  \n- **Multi-Agent Discussion & Bargaining**: Modular architectures that negotiate responsibilities in negotiation settings.  \n- **Delegation Mechanisms**: Task- or responsibility-delegation techniques tightly coupled with system design principles.  \n(Data: 70, 78, 71)\n\n## 4. Feedback-Driven Optimization  \nEmploys iterative loops—both automated and human-in-the-loop—to refine agent performance.  \n- **Trial-and-Error Exploration**: Iteratively tests prompt/agent variants to identify high-performing patterns.  \n- **Error Feedback**: Uses model responses or crowd-sourced reports to generate corrective signals.  \n- **Feedback-Driven Reprompting**: Automated critique loops where agents refine each other’s outputs via targeted re-prompts.  \n(Data: 1, 7, 160)\n\n## 5. Architectural Patterns & Tool Integration  \nDefines structural blueprints and external tool calls to extend agent capabilities.  \n- **Cooperative Multi-Agent Teams**: Specialized agents (e.g., geographic data vs. cost optimization) collaborating for robustness.  \n- **Tool-Calling Agents**: Embeds calls to external services (databases, calculators) within prompts to share context.  \n- **QA-Driven ABM Pipelines**: Automated generation of agent-based models via question-answering loops.  \n(Data: 163, 164, 14, 78)\n\n## 6. Domain-Specific Systems (Narrative & Simulation)  \nApplies MAS methods to specialized domains such as storytelling or field validation.  \n- **Companion Development**: LLM “companions” that suggest narrative twists and maintain character consistency.  \n- **Mood Systems**: Sentiment-tracking architectures to adapt agents’ tone and emotional state over time.  \n- **Inter-Agent Protocols**: Centralized vs. decentralized control, communication protocols, and conflict-resolution in simulated environments.  \n(Data: 191, 192, 188)\n\n---\n\n**Implications:**  \nBy combining these methods, researchers can design MAS that balance individual agent autonomy with global coherence, continuously refine performance via feedback loops, and leverage specialized architectures for domain-specific tasks. Continuous integration of prompt engineering, systematic feedback, and robust coordination protocols remains at the forefront of MAS innovation.'")

'# Common Methods in Multi-Agent Systems Research

Synthesizing insights across multiple analyst reports, we identify six principal families of methods employed in multi-agent systems (MAS) papers. Each reflects distinct goals—ranging from prompt coordination and optimization loops to architectural patterns and domain-specific systems.

---

## 1. Prompt Engineering & In-Context Learning  
Rigorously defines how agents generate, structure, and share prompts to coordinate actions.  
- **Formal Prompt Engineering Framework**: Specifies prompt spaces, constraints, and coordination objectives.  
- **Multi-Objective Directional Prompting**: Guides each agent toward goals (e.g., accuracy, style) via directional vectors in prompt space.  
- **Dynamic Prompt Assembly**: Constructs modular “sub-agent” prompts at inference time based on feedback and roles.  
(Data: 7, 14, 117)

## 2. Chain-of-Thought Reasoning & Reflection  
Structures complex decision processes and self-evaluation across collab

#### Global question generator 

In [5]:
from src.global_question_generator import GlobalQuestionGenerator

In [6]:
corpus_description = """ 
This corpus contains 20 recent research papers retrieved from arXiv using the query: "agent OR large language model OR prompt engineering". The papers focus on cutting-edge advancements in large language models (LLMs), intelligent agent architectures, and prompt engineering strategies to enhance model reasoning and coordination. Key areas include:

- Design of multi-agent systems using LLMs as agents or controllers.
- Prompting techniques such as Chain-of-Thought, Reflection, and ReAct used for structured reasoning.
- Methods for orchestrating collaborative agents in planning, execution, or communication tasks.
- Evaluation of LLM interpretability, alignment with human intent, and safe deployment in dynamic environments.
- Societal impacts, including ethical considerations, safety risks, and governance frameworks for autonomous LLM-based systems.

The corpus spans technical proposals, system frameworks, empirical evaluations, and conceptual discussions relevant to both applied NLP and AI safety domains.
"""

generator = GlobalQuestionGenerator(
    model="o4-mini-2025-04-16",
    output_path="./data/global_questions/t20"
)

questions = generator.run(
    corpus_description=corpus_description,
    k=2,
    n=2,
    m=2
)

🚀 Starting global question generation...
📦 Target: 2 personas × 2 tasks × 2 questions = 8 total questions

👤 Generating tasks for Persona 1/2...
   📌 Generating 2 questions for Task 1/2...
     ✅ Progress: 2/8 questions generated

👤 Generating tasks for Persona 2/2...
   📌 Generating 2 questions for Task 1/2...
     ✅ Progress: 4/8 questions generated
   📌 Generating 2 questions for Task 2/2...
     ✅ Progress: 6/8 questions generated

💾 Saving generated questions...
✅ Saved 3 persona-task blocks to ./data/global_questions/t20/global_sensemaking_questions.json
✅ All questions saved successfully.


#### Sensemaking avaluator 

In [1]:
from src.sensemaking_evaluator import SensemakingEvaluator

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.0 at grpc_health/v1/health.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.0 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.0 at v1/base.proto. Please update the genc

In [2]:
from src.rag_pipeline import RAGPipeline

config = {
    "mode": "load",
    "kg_db_name": "t20",
    "summary_dir": "/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C3",
}

pipeline = RAGPipeline(config=config)
pipeline.initialize()

evaluator = SensemakingEvaluator(
    question_path="./data/global_questions/t20/global_sensemaking_questions.json",
    output_path="./data/global_questions/t20/C3_evaluated_answers.md",
    rag_pipeline=pipeline,
    eval_model="o4-mini-2025-04-16"
)

evaluator.evaluate()

Milvus DB exists. Connecting to existing database...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/milvus_lite/__init__.py:15: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import DistributionNotFound, get_distribution


🔍 Evaluating Q1/10
🧩 Prepared 30 summary chunks.
✅ Collected 27 helpful answers.
🔍 Evaluating Q2/10
🧩 Prepared 30 summary chunks.
✅ Collected 27 helpful answers.
🔍 Evaluating Q3/10
🧩 Prepared 30 summary chunks.
✅ Collected 27 helpful answers.
🔍 Evaluating Q4/10
🧩 Prepared 30 summary chunks.
✅ Collected 23 helpful answers.
🔍 Evaluating Q5/10
🧩 Prepared 30 summary chunks.
✅ Collected 29 helpful answers.
🔍 Evaluating Q6/10
🧩 Prepared 30 summary chunks.
✅ Collected 25 helpful answers.
🔍 Evaluating Q7/10
🧩 Prepared 30 summary chunks.
✅ Collected 29 helpful answers.
🔍 Evaluating Q8/10
🧩 Prepared 30 summary chunks.
✅ Collected 22 helpful answers.
🔍 Evaluating Q9/10
🧩 Prepared 30 summary chunks.
✅ Collected 28 helpful answers.
🔍 Evaluating Q10/10
🧩 Prepared 30 summary chunks.
✅ Collected 29 helpful answers.

✅ All evaluations saved to: ./data/global_questions/t20/C3_evaluated_answers.md


In [3]:
from src.rag_pipeline import RAGPipeline

config = {
    "mode": "load",
    "kg_db_name": "t20",
    "summary_dir": "/Users/danielguarnizo/workspace/Master/LLMs/GraphRAG/data/community_summaries/t20/C2",
}

pipeline = RAGPipeline(config=config)
pipeline.initialize()

evaluator = SensemakingEvaluator(
    question_path="./data/global_questions/t20/global_sensemaking_questions.json",
    output_path="./data/global_questions/t20/C2_evaluated_answers.md",
    rag_pipeline=pipeline,
    eval_model="o4-mini-2025-04-16"
)

evaluator.evaluate()

Milvus DB exists. Connecting to existing database...
🔍 Evaluating Q1/10
🧩 Prepared 27 summary chunks.
✅ Collected 25 helpful answers.
🔍 Evaluating Q2/10
🧩 Prepared 27 summary chunks.
✅ Collected 23 helpful answers.
🔍 Evaluating Q3/10
🧩 Prepared 27 summary chunks.
✅ Collected 24 helpful answers.
🔍 Evaluating Q4/10
🧩 Prepared 27 summary chunks.
✅ Collected 24 helpful answers.
🔍 Evaluating Q5/10
🧩 Prepared 27 summary chunks.
✅ Collected 25 helpful answers.
🔍 Evaluating Q6/10
🧩 Prepared 27 summary chunks.
✅ Collected 22 helpful answers.
🔍 Evaluating Q7/10
🧩 Prepared 27 summary chunks.
✅ Collected 24 helpful answers.
🔍 Evaluating Q8/10
🧩 Prepared 27 summary chunks.
✅ Collected 20 helpful answers.
🔍 Evaluating Q9/10
🧩 Prepared 27 summary chunks.
✅ Collected 25 helpful answers.
🔍 Evaluating Q10/10
🧩 Prepared 27 summary chunks.
✅ Collected 24 helpful answers.

✅ All evaluations saved to: ./data/global_questions/t20/C2_evaluated_answers.md
